# 特征数据的处理

特征数据是由原始数据经过处理的得来的用于模型训练.模型预测,数据分析,数据挖掘的一类特殊数据.在万物靠算法的今天重要性也越来越高.特征数据的存储和管理也早已经是工程界的一个课题.

特征数据有如下几个特点:

+ 时效性,很多时候时间本身就是一个特征,做过模型开发的同学肯定都知道拿当前的数据训练预测过去的结果是不可行的,因此特征都是有时效性的,这决定了特征数据是一个时间序列数据.一般会分两个列用于记录特征发生时间和创建时间,他们一般是datetime类型,也有用int64放时间戳或者用string放特定格式时间字符串的.
+ 结构化,现在的机器学习模型可以只加载原始数据,但如果要用特征,特征数据往往都是结构化的,而且数据类型相对比较固定,大致可以分为:
    + 标量数据,也就是一列只有一个值的数据,通常在传统机器学习模型,数据挖掘,数据分析场景下用的比较多,细分类型可以分为:
        + 数值型:一般是double或者float32类型,也有int32或者int64类型的但不多.这类用于描述连续数值.比如描述植株花,叶的尺寸的特征.
        + 分类型:一般是string类型,也有用int32类型或者二分类用bool型的.这类用于描述离散的分类信息,比如手机品牌,用户所在城市这类特征.
    + 向量数据,也就是一列保存一个向量的数据,通常这个向量中元素类型都是float32或者double.这类特征通常由模型或算法得到,比如词向量,bert向量.这类特征已经广泛用于深度学习模型.向量数据很多时候在推理侧还需要可以搜索出最临近的top-k,比如在相似推荐场景,这也是它的特殊性之一.
+ 可截取.训练侧需要可以根据时间戳提取出当时的特征数据避免未来数据的引入造成模型是真,而推理侧需要保持数据为最近截面,因此特征数据需要可按时间和时效性截取
+ 可拼接,生产环境下通常特征工程并不是一次都做出来的,而是分批做出来的,而我们无论是做数据分析,数据挖掘还是构造模型基本不可能只用同一批构造出来的特征,因此特征必须可拼接.特征拼接的轴一般是2个
    + 时效,通常是特征的发生时间向前追溯一个特定的时间间隔,取这个间隔内最近的一条符合条件的数据
    + 实体,就是这批特征描述的对象

而特征的存储也有如下特点:

+ 需要多份保存.通常特征在训练侧是批量抽取,要求的是时间跨度大覆盖面全,对于抽取时间一般没有特别高的要求,但在推理侧正好相反,是单条抽取或小批抽取,要求的是延迟低,抽取到的特征需要保证在时效内.不同的需求必然导致我们不得不在不同的工具中多分存储.通常训练侧数据会保存在关系数据库,列存储的分布式数据库,或者直接以文件的形式用parquet格式按时间顺序全量保存在nfs或者oss中;而推理侧一般则用redis保存数据且只保存最有时效性的最近切面数据.

+ 需要特殊的更新逻辑.特征根据时效性和使用的场景,它的更新逻辑也是不一样的.具体来看:
    + 训练侧不能更新数据状态,只能只增的描述记录下更新时刻的数据
    + 推理侧则必须更新数据状态,并且如果数据超过了时效时间应当可以失效


## 特征数据的管理工具

特征数据管理工具目前开源的比较活跃的应该只有[feast](https://github.com/feast-dev/feast)

### feast

feast是一个专注于特征管理的python包,另外额外提供docker镜像用于推理侧获取特征数据.总体来说这个项目的完成度并不算特别高,但也已经是生产环境可用的状态了.

feast经历过一次重大改版,在0.10之前feast是一套特征管理服务,后来可能意识到铺的太大了就经历了一次重大改版,缩减成了一个python包,但目前看似乎还是铺的太大了.总体来说feast侧重推理侧,在训练侧支持对接的数据库不算多,功能很多也没做完,基本上比较可用的组合是:

+ 一个nfs存放声明特征和配置用的文件夹
+ 仓库元数据使用file模式
+ 推理侧对接redis
+ 训练侧对接file,从s3(minio)上读数据

参考如下docker-compose部署一个本地的测试环境:

+ docker-compose.yml




+ 